# Import Libraries

In [108]:
import time
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from deep_translator import GoogleTranslator
import emoji
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.utils.dediac import dediac_ar
import os
import sys
import textwrap
from pathlib import Path
from dotenv import load_dotenv
import requests
import json


# Twitter Scraping

In [ ]:
SINCE_TOTAL = '2025-06-01'
UNTILL_TOTAL = '2025-09-18'
LANG = "ar"
PERIOD_DAYS = 1

EMAIL = "email"
MY_USER = "user"
PASSWORD = "password"

start_date = datetime.strptime(SINCE_TOTAL, "%Y-%m-%d")
end_date = datetime.strptime(UNTILL_TOTAL, "%Y-%m-%d")

periods = []
while start_date < end_date:
    period_end = min(start_date + timedelta(days=PERIOD_DAYS), end_date)
    periods.append((start_date.strftime("%Y-%m-%d"), period_end.strftime("%Y-%m-%d")))
    start_date = period_end


In [ ]:
options = Options()
options.add_experimental_option("detach", True)

options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)
driver.get("https://x.com/i/flow/login")

time.sleep(2)

In [ ]:
driver.find_element(By.XPATH,
                    '/html/body/div/div/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/div[4]/label/div/div[2]/div/input').send_keys(
    EMAIL)
driver.find_element(By.XPATH,
                    '/html/body/div/div/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/button[2]').click()
time.sleep(5)
driver.find_element(By.CSS_SELECTOR, 'input[data-testid="ocfEnterTextTextInput"]').send_keys(MY_USER)
driver.find_element(By.XPATH, '//span[text()="Next"]').click()
time.sleep(5)
driver.find_element(By.NAME, "password").send_keys(PASSWORD)
driver.find_element(By.CSS_SELECTOR, "button[data-testid='LoginForm_Login_Button']").click()
time.sleep(5)

In [ ]:
all_tweets = []
pbar = tqdm(desc="Tweets collected", unit=" tweet")

for since, until in periods:
    url = format(f"https://x.com/search?q=lang%3Aar%20until%3A{until}%20since%3A{since}%20-filter%3Alinks%20-filter%3Areplies&src=typed_query")
    driver.get(url)
    time.sleep(5)

    last_height = driver.execute_script("return document.body.scrollHeight")
    tweets_data = []
    while True:
        tweet_cards = driver.find_elements(By.CSS_SELECTOR, "article[data-testid='tweet']")
        try:
            retry_button = driver.find_element(By.XPATH, '//span[text()="Retry"]/ancestor::button')
            retry_button.click()
            time.sleep(30)
            continue
        except:
            pass
        for tweet in tweet_cards:
            user_name = tweet.find_element(By.CSS_SELECTOR, "div[data-testid='User-Name'] span").text
            username = tweet.find_element(By.CSS_SELECTOR, "div.r-1d09ksm a span").text
            text = tweet.find_element(By.CSS_SELECTOR, "div[data-testid='tweetText']").text
            time_posted = tweet.find_element(By.TAG_NAME, "time").get_attribute("datetime")

            tweet_info = {
                "user_name": user_name,
                "username": username,
                "text": text,
                "time_posted": time_posted,
            }

            if tweet_info not in all_tweets:
                all_tweets.append(tweet_info)
                pbar.update(1)

        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            df = pd.DataFrame(all_tweets)
            df.to_csv("X2.csv", index=False, mode='w')
            time.sleep(2)
            break
        last_height = new_height

pbar.close()
driver.quit()

# Load Data

In [ ]:
df = pd.read_csv("/content/X2.csv", sep=",")

# Data Preprocessing

In [ ]:
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')
ar_stopwords = set(stopwords.words('arabic'))
en_stopwords = set(stopwords.words('english'))
en_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df.head()

,user_name,username,text,time_posted
0,بيبرس,بيبرس,﴿وَاسْتَغْفِرِ اللهَ إِنَّ اللهَ كَانَ غَفُورً...,2025-06-01T23:59:35.000Z
1,عبدالله البرقاوي,عبدالله البرقاوي,ويبقى الصباح كساعي البريد إلى أهل الصفاء والنق...,2025-06-01T23:59:44.000Z
2,عماد عبدالقادر,عماد عبدالقادر,توقعوا\nعن يقين مطلق بالله القاهر فوق عباده با...,2025-06-01T23:59:41.000Z
3,النَبٰيُـــــــ♡ـــــــلّة,النَبٰيُـــــــ♡ـــــــلّة,« تذكروا أنّ أُنس الأموات الدعاء »\n\nاللهم ار...,2025-06-01T23:59:32.000Z
4,المعلم رشدان,المعلم رشدان,انا بكره الفرانكو واللي بيكتبو فرانكو ليه مصري...,2025-06-01T23:59:48.000Z


In [ ]:
df.shape

(9756, 4)

In [ ]:
df.isnull().sum()

,0
user_name,470
username,470
text,0
time_posted,0


In [ ]:
def replace_emoj(sen):
  n = ""
  for word in sen.split():
    for w in word:
      if emoji.is_emoji(w):
        w = emoji.demojize(w)
        w = re.sub(r'[_:-]', ' ', w)
        w = GoogleTranslator(source='en', target='ar').translate(w)
        n += w
      else:
        n += word + " "
        break
  return n

In [ ]:
def handle_ar(sen):
  sen = re.sub(r'[إأآا]', 'ا', sen)
  sen = re.sub(r'ة', 'ه', sen)
  sen = re.sub(r'[ى]', 'ي', sen)
  sen = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', sen)
  return sen

In [ ]:
def handle_en(sen):
  sen = sen.lower()
  sen = nltk.word_tokenize(sen)
  sen = [en_lemmatizer.lemmatize(token.lower()) for token in sen if token.lower() not in en_stopwords]
  return " ".join(sen)

In [ ]:
def preprocess(sen):
    sen = re.sub(r'http\S+|www\.\S+|\.com', ' ', sen)
    sen = re.sub(r'@\w+|#\w+', ' ', sen)
    sen = re.sub(r'(.)\1{2,}', r'\1', sen)
    sen = re.sub(r'\s+', ' ', sen).strip()
    sen = replace_emoj(sen)
    sen = re.sub(r'[^\w\s\u0600-\u06FF0-9]', '', sen)
    if re.search(r"[a-zA-Z]", sen):
      sen = handle_en(sen)
    elif re.search(r'[\u0600-\u06FF]', sen):
      sen = handle_ar(sen)
    return sen

In [ ]:
df['text'] = df['text'].apply(lambda x: preprocess(x))

In [ ]:
df.head()

,user_name,username,text,time_posted
0,بيبرس,بيبرس,واستغفر الله ان الله كان غفورا رحيما,2025-06-01T23:59:35.000Z
1,عبدالله البرقاوي,عبدالله البرقاوي,ويبقي الصباح كساعي البريد الي اهل الصفاء والنق...,2025-06-01T23:59:44.000Z
2,عماد عبدالقادر,عماد عبدالقادر,توقعوا عن يقين مطلق بالله القاهر فوق عباده بان...,2025-06-01T23:59:41.000Z
3,النَبٰيُـــــــ♡ـــــــلّة,النَبٰيُـــــــ♡ـــــــلّة,تذكروا ان انس الاموات الدعاء اللهم ارحم موتا...,2025-06-01T23:59:32.000Z
4,المعلم رشدان,المعلم رشدان,انا بكره الفرانكو واللي بيكتبو فرانكو ليه مصري...,2025-06-01T23:59:48.000Z


In [ ]:
df['time_posted'] = pd.to_datetime(df['time_posted'])

df['date'] = df['time_posted'].dt.date
df['time'] = df['time_posted'].dt.time

In [ ]:
df.head()

,user_name,username,text,time_posted,date,time
0,بيبرس,بيبرس,واستغفر الله ان الله كان غفورا رحيما,2025-06-01 23:59:35+00:00,2025-06-01,23:59:35
1,عبدالله البرقاوي,عبدالله البرقاوي,ويبقي الصباح كساعي البريد الي اهل الصفاء والنق...,2025-06-01 23:59:44+00:00,2025-06-01,23:59:44
2,عماد عبدالقادر,عماد عبدالقادر,توقعوا عن يقين مطلق بالله القاهر فوق عباده بان...,2025-06-01 23:59:41+00:00,2025-06-01,23:59:41
3,النَبٰيُـــــــ♡ـــــــلّة,النَبٰيُـــــــ♡ـــــــلّة,تذكروا ان انس الاموات الدعاء اللهم ارحم موتا...,2025-06-01 23:59:32+00:00,2025-06-01,23:59:32
4,المعلم رشدان,المعلم رشدان,انا بكره الفرانكو واللي بيكتبو فرانكو ليه مصري...,2025-06-01 23:59:48+00:00,2025-06-01,23:59:48


# Build Gemini

In [ ]:
API_KEY = "API"
MODEL_NAME = "gemini-1.5-flash"
API_URL = f"https://generativelanguage.googleapis.com/v1beta/models/{MODEL_NAME}:generateContent"

In [ ]:
def build_system_prompt(tweet):
    prompt = textwrap.dedent(f"""
    Analyze the following tweet and provide:
    1. Content category: sexual, political, terrorist, hateful, scam, normal, other
    2. Sentiment: useful, harmful, neutral
    3. Named entities (persons, organizations, locations, dates...)

    Tweet: "{tweet}"

    Output format:
    - Content Category: Just Determined Category .
    - Sentiment: ... Just Determined Class .
    - Named Entities: [...] if Not fount return None .
    """).strip()
    return prompt

In [ ]:
def call_gemini(prompt, temperature=0.0, max_output_tokens=512):
    headers = {"Content-Type": "application/json"}
    payload = {
        "contents": [
            {"parts": [{"text": prompt}]}
        ],
        "generationConfig": {
            "temperature": temperature,
            "maxOutputTokens": max_output_tokens
        }
    }
    params = {"key": API_KEY}

    resp = requests.post(API_URL, params=params, headers=headers, json=payload, timeout=60)
    if not resp.ok:
        raise RuntimeError(f"Generative API error {resp.status_code}: {resp.text}")

    data = resp.json()
    try:
        return data["candidates"][0]["content"]["parts"][0]["text"]
    except Exception:
        return json.dumps(data, indent=2, ensure_ascii=False)

# Tweets Moderation

In [ ]:
def moderate_tweet(user, tweet):
  prompt = build_system_prompt(tweet)
  output = call_gemini(prompt, temperature=0.2)
  content_cat = re.findall(r"Content Category:\s*(.*)[^\.]", output)
  sentiment = re.findall(r"Sentiment:\s*(.*)[^\.]", output)
  NER = re.findall(r"Named Entities:\s*(.*)[^\.]", output)
  data = {"user": user, "tweet": tweet, "content category": content_cat, "sentiment": sentiment, "named entities": NER}
  return pd.DataFrame(data)

In [ ]:
moderate_tweet("user", "القتل والتفجيرات في القاهره المدينة")

,user,tweet,content category,sentiment,named entities
0,user,القتل والتفجيرات في القاهره المدينة,terrorist .,harmful .,[Cairo] .


In [ ]:

df_sample = df.sample(1).copy()
results = []
df_new = pd.DataFrame(results)
for tweet in df_sample:
    results = moderate_tweet(df_sample['user_name'], df_sample['text'])
    time.sleep(5)

In [ ]:
results

,user,tweet,content category,sentiment,named entities
7520,لذيذ ‘,في جاد الحين يبي امصله اموت بالمص,sexual,harmful,None
